### 1. Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

### 2.  Data processing

### 2.1 read in data

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [6]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

### 2.2 Feature selection: excluding columns of 'RowNumber', 'CustomerId', 'Surname'

In [8]:
X = dataset.iloc[:, 3:13].values

In [9]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [10]:
y = dataset.iloc[:, 13].values

In [11]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

### 2.3 Categorical encoding: column Geography and Gender

In [12]:
#Label encoder for ''Geography'', 1 is the index of 'country' col
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

In [13]:
X

array([[619, 0, 'Female', ..., 1, 1, 101348.88],
       [608, 2, 'Female', ..., 0, 1, 112542.58],
       [502, 0, 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 0, 'Female', ..., 0, 1, 42085.58],
       [772, 1, 'Male', ..., 1, 0, 92888.52],
       [792, 0, 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [14]:
#Label encoder for 'Gender'
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

#### remove genography column

In [15]:
X_temp = np.delete(X, 1, 1)
X_temp

array([[619, 0, 42, ..., 1, 1, 101348.88],
       [608, 0, 41, ..., 0, 1, 112542.58],
       [502, 0, 42, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 36, ..., 0, 1, 42085.58],
       [772, 1, 42, ..., 1, 0, 92888.52],
       [792, 0, 28, ..., 1, 0, 38190.78]], dtype=object)

In [16]:
1/0

ZeroDivisionError: division by zero

### 2.4 One-hot encoding: column Geography 

In [17]:
#One-hot encoding ''Geography'' col
#One-hot encoding 'country' col
onehotencoder = OneHotEncoder()
geography_array = onehotencoder.fit_transform(X[:, 1:2]).toarray()

### 2.5 avoid dummy data trap

In [18]:
geography_array = geography_array[: , 1:3]
geography_array

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       ...,
       [0., 0.],
       [1., 0.],
       [0., 0.]])

In [19]:
X_final = np.concatenate( (geography_array, X_temp),  axis = 1)
X_final

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

### 2.6 Data split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size = 0.2, random_state = 0)

### 2.7 Feature scaling

In [21]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### 3. K-fold cross validation

In [22]:
##============evalate the model using k-fold cross validatioN
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 50)

In [24]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
7200/7200 [==============================] - 2s 219us/step - loss: 0.4874 - acc: 0.7967
Epoch 2/100
7200/7200 [==============================] - 1s 167us/step - loss: 0.4299 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 197us/step - loss: 0.4261 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 165us/step - loss: 0.4219 - acc: 0.7976 0s - loss: 0.4235 - acc: 0.79
Epoch 5/100
7200/7200 [==============================] - 1s 137us/step - loss: 0.4192 - acc: 0.8167
Epoch 6/100
7200/7200 [==============================] - 1s 126us/step - loss: 0.4173 - acc: 0.8217
Epoch 7/100
7200/7200 [==============================] - 1s 123us/step - loss: 0.4152 - acc: 0.8268
Epoch 8/100
7200/7200 [==============================] - 1s 125us/step - loss: 0.4140 - acc: 0.8306
Epoch 9/100
7200/7200 [==============================] - 1s 140us/step

7200/7200 [==============================] - 1s 190us/step - loss: 0.3997 - acc: 0.8336
Epoch 65/100
7200/7200 [==============================] - 1s 142us/step - loss: 0.3998 - acc: 0.8360
Epoch 66/100
7200/7200 [==============================] - 1s 168us/step - loss: 0.3994 - acc: 0.8354
Epoch 67/100
7200/7200 [==============================] - 1s 190us/step - loss: 0.3999 - acc: 0.8349
Epoch 68/100
7200/7200 [==============================] - 1s 188us/step - loss: 0.3993 - acc: 0.8356
Epoch 69/100
7200/7200 [==============================] - 1s 194us/step - loss: 0.3994 - acc: 0.8351
Epoch 70/100
7200/7200 [==============================] - 1s 180us/step - loss: 0.3992 - acc: 0.8358
Epoch 71/100
7200/7200 [==============================] - 1s 153us/step - loss: 0.3993 - acc: 0.8356
Epoch 72/100
7200/7200 [==============================] - 1s 188us/step - loss: 0.3995 - acc: 0.8344
Epoch 73/100
7200/7200 [==============================] - 2s 220us/step - loss: 0.3992 - acc: 0.8343
Epo

7200/7200 [==============================] - 2s 243us/step - loss: 0.3464 - acc: 0.8593
Epoch 45/100
7200/7200 [==============================] - 1s 202us/step - loss: 0.3456 - acc: 0.8610
Epoch 46/100
7200/7200 [==============================] - 1s 197us/step - loss: 0.3455 - acc: 0.8600
Epoch 47/100
7200/7200 [==============================] - 2s 233us/step - loss: 0.3455 - acc: 0.8596
Epoch 48/100
7200/7200 [==============================] - 1s 167us/step - loss: 0.3446 - acc: 0.8612
Epoch 49/100
7200/7200 [==============================] - 1s 142us/step - loss: 0.3460 - acc: 0.8593
Epoch 50/100
7200/7200 [==============================] - 1s 134us/step - loss: 0.3438 - acc: 0.8608
Epoch 51/100
7200/7200 [==============================] - 1s 133us/step - loss: 0.3447 - acc: 0.8582
Epoch 52/100
7200/7200 [==============================] - 1s 138us/step - loss: 0.3423 - acc: 0.8592
Epoch 53/100
7200/7200 [==============================] - ETA: 0s - loss: 0.3439 - acc: 0.860 - 1s 138us

7200/7200 [==============================] - 1s 148us/step - loss: 0.3942 - acc: 0.8349
Epoch 23/100
7200/7200 [==============================] - 1s 148us/step - loss: 0.3933 - acc: 0.8362
Epoch 24/100
7200/7200 [==============================] - 1s 142us/step - loss: 0.3937 - acc: 0.8369
Epoch 25/100
7200/7200 [==============================] - 1s 175us/step - loss: 0.3929 - acc: 0.8362
Epoch 26/100
7200/7200 [==============================] - 2s 211us/step - loss: 0.3918 - acc: 0.8386
Epoch 27/100
7200/7200 [==============================] - 2s 209us/step - loss: 0.3906 - acc: 0.8364
Epoch 28/100
7200/7200 [==============================] - 1s 167us/step - loss: 0.3889 - acc: 0.8354
Epoch 29/100
7200/7200 [==============================] - 1s 141us/step - loss: 0.3868 - acc: 0.8382
Epoch 30/100
7200/7200 [==============================] - 1s 151us/step - loss: 0.3833 - acc: 0.8386
Epoch 31/100
7200/7200 [==============================] - 1s 153us/step - loss: 0.3813 - acc: 0.8378
Epo

7200/7200 [==============================] - 1s 171us/step - loss: 0.4236 - acc: 0.7975
Epoch 3/100
7200/7200 [==============================] - 1s 166us/step - loss: 0.4180 - acc: 0.8089
Epoch 4/100
7200/7200 [==============================] - 1s 161us/step - loss: 0.4151 - acc: 0.8251
Epoch 5/100
7200/7200 [==============================] - 1s 153us/step - loss: 0.4133 - acc: 0.8290
Epoch 6/100
7200/7200 [==============================] - 1s 156us/step - loss: 0.4118 - acc: 0.8328
Epoch 7/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4102 - acc: 0.8336
Epoch 8/100
7200/7200 [==============================] - 2s 210us/step - loss: 0.4090 - acc: 0.8349
Epoch 9/100
7200/7200 [==============================] - 1s 175us/step - loss: 0.4078 - acc: 0.8349 2s - lo
Epoch 10/100
7200/7200 [==============================] - 1s 144us/step - loss: 0.4071 - acc: 0.8354
Epoch 11/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4064 - acc: 0.8351
Ep

...

7200/7200 [==============================] - 1s 156us/step - loss: 0.4041 - acc: 0.8342
Epoch 24/100
7200/7200 [==============================] - 1s 158us/step - loss: 0.4040 - acc: 0.8353
Epoch 25/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.4036 - acc: 0.8350
Epoch 26/100
7200/7200 [==============================] - 2s 236us/step - loss: 0.4037 - acc: 0.8369
Epoch 27/100
7200/7200 [==============================] - 1s 178us/step - loss: 0.4033 - acc: 0.8361
Epoch 28/100
7200/7200 [==============================] - 1s 156us/step - loss: 0.4031 - acc: 0.8365
Epoch 29/100
7200/7200 [==============================] - 1s 148us/step - loss: 0.4032 - acc: 0.8358
Epoch 30/100
7200/7200 [==============================] - 1s 148us/step - loss: 0.4030 - acc: 0.8349
Epoch 31/100
7200/7200 [==============================] - 1s 152us/step - loss: 0.4021 - acc: 0.8346
Epoch 32/100
7200/7200 [==============================] - 1s 164us/step - loss: 0.4026 - acc: 0.8346
Epo

7200/7200 [==============================] - 1s 153us/step - loss: 0.4214 - acc: 0.7962
Epoch 4/100
7200/7200 [==============================] - 1s 153us/step - loss: 0.4171 - acc: 0.8237
Epoch 5/100
7200/7200 [==============================] - 1s 155us/step - loss: 0.4146 - acc: 0.8289
Epoch 6/100
7200/7200 [==============================] - 1s 157us/step - loss: 0.4130 - acc: 0.8307
Epoch 7/100
7200/7200 [==============================] - 1s 170us/step - loss: 0.4116 - acc: 0.8325
Epoch 8/100
7200/7200 [==============================] - 1s 158us/step - loss: 0.4103 - acc: 0.8340
Epoch 9/100
7200/7200 [==============================] - 1s 156us/step - loss: 0.4095 - acc: 0.8329
Epoch 10/100
7200/7200 [==============================] - 1s 154us/step - loss: 0.4084 - acc: 0.8339
Epoch 11/100
7200/7200 [==============================] - 1s 162us/step - loss: 0.4080 - acc: 0.8349
Epoch 12/100
7200/7200 [==============================] - 1s 183us/step - loss: 0.4069 - acc: 0.8361
Epoch 13/

7200/7200 [==============================] - 1s 164us/step - loss: 0.3981 - acc: 0.8378
Epoch 85/100
7200/7200 [==============================] - 1s 156us/step - loss: 0.3985 - acc: 0.8357
Epoch 86/100
7200/7200 [==============================] - 1s 161us/step - loss: 0.3984 - acc: 0.8351
Epoch 87/100
7200/7200 [==============================] - 1s 160us/step - loss: 0.3985 - acc: 0.8360
Epoch 88/100
7200/7200 [==============================] - 1s 157us/step - loss: 0.3986 - acc: 0.8369
Epoch 89/100
7200/7200 [==============================] - 2s 228us/step - loss: 0.3984 - acc: 0.8374
Epoch 90/100
7200/7200 [==============================] - 2s 222us/step - loss: 0.3979 - acc: 0.8371
Epoch 91/100
7200/7200 [==============================] - 2s 232us/step - loss: 0.3983 - acc: 0.8365
Epoch 92/100
7200/7200 [==============================] - 3s 399us/step - loss: 0.3981 - acc: 0.8371
Epoch 93/100
7200/7200 [==============================] - 2s 260us/step - loss: 0.3977 - acc: 0.8350
Epo

7200/7200 [==============================] - 2s 230us/step - loss: 0.3979 - acc: 0.8356
Epoch 65/100
7200/7200 [==============================] - 2s 241us/step - loss: 0.3972 - acc: 0.8351
Epoch 66/100
7200/7200 [==============================] - 2s 241us/step - loss: 0.3975 - acc: 0.8349
Epoch 67/100
7200/7200 [==============================] - 2s 233us/step - loss: 0.3973 - acc: 0.8362
Epoch 68/100
7200/7200 [==============================] - 2s 243us/step - loss: 0.3974 - acc: 0.8369
Epoch 69/100
7200/7200 [==============================] - 2s 238us/step - loss: 0.3976 - acc: 0.8361
Epoch 70/100
7200/7200 [==============================] - 2s 236us/step - loss: 0.3973 - acc: 0.8357
Epoch 71/100
7200/7200 [==============================] - 2s 241us/step - loss: 0.3966 - acc: 0.8367
Epoch 72/100
7200/7200 [==============================] - 2s 233us/step - loss: 0.3977 - acc: 0.8349
Epoch 73/100
7200/7200 [==============================] - 2s 241us/step - loss: 0.3968 - acc: 0.8350
Epo

7200/7200 [==============================] - 1s 163us/step - loss: 0.4039 - acc: 0.8328
Epoch 45/100
7200/7200 [==============================] - 1s 187us/step - loss: 0.4040 - acc: 0.8347
Epoch 46/100
7200/7200 [==============================] - 1s 162us/step - loss: 0.4045 - acc: 0.8347
Epoch 47/100
7200/7200 [==============================] - 1s 185us/step - loss: 0.4037 - acc: 0.8342
Epoch 48/100
7200/7200 [==============================] - 1s 196us/step - loss: 0.4039 - acc: 0.8343
Epoch 49/100
7200/7200 [==============================] - 1s 162us/step - loss: 0.4037 - acc: 0.8321
Epoch 50/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.4039 - acc: 0.8325
Epoch 51/100
7200/7200 [==============================] - 1s 159us/step - loss: 0.4037 - acc: 0.8344
Epoch 52/100
7200/7200 [==============================] - 1s 158us/step - loss: 0.4038 - acc: 0.8342
Epoch 53/100
7200/7200 [==============================] - 1s 173us/step - loss: 0.4034 - acc: 0.8328
Epo

In [25]:
mean = accuracies.mean()
print('Average accuracy during cross validation: ', mean) 

Average accuracy during cross validation:  0.8434999949485062


In [26]:
accuracies

array([0.8375    , 0.86124999, 0.88624999, 0.82625   , 0.85249999,
       0.85875   , 0.83125   , 0.8225    , 0.815     , 0.84374999])

In [27]:
std = accuracies.std()
print('Accuracy std cross validation: ', std) 

Accuracy std cross validation:  0.02047712063174261


## Parameter tuning

In [28]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [29]:
classifier = KerasClassifier(build_fn = build_classifier)

In [30]:
#define parameter to optimize
#dict key needs to be the same name as defined in classifer
parameters = {'batch_size': [25, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

In [31]:
grid_search = GridSearchCV(estimator = classifier,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 10)

In [ ]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/100
7200/7200 [==============================] - 1s 178us/step - loss: 0.5552 - acc: 0.7969
Epoch 2/100
7200/7200 [==============================] - 0s 66us/step - loss: 0.4366 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 69us/step - loss: 0.4316 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 74us/step - loss: 0.4292 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.4268 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.4241 - acc: 0.7971
Epoch 7/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.4213 - acc: 0.7971
Epoch 8/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.4192 - acc: 0.8157
Epoch 9/100
7200/7200 [==============================] - 0s 66us/step - loss: 0.4176 - acc: 0.8206
Epoch 10/100
7200/7200 [==============================] - 0s 68us/step - loss: 0.4165 - acc: 0.8243
Epoch 11

7200/7200 [==============================] - 1s 70us/step - loss: 0.4011 - acc: 0.8343
Epoch 83/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4013 - acc: 0.8349
Epoch 84/100
7200/7200 [==============================] - 0s 67us/step - loss: 0.4010 - acc: 0.8351
Epoch 85/100
7200/7200 [==============================] - 1s 74us/step - loss: 0.4010 - acc: 0.8357
Epoch 86/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.4011 - acc: 0.8350
Epoch 87/100
7200/7200 [==============================] - 1s 74us/step - loss: 0.4007 - acc: 0.8356
Epoch 88/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4010 - acc: 0.8354
Epoch 89/100
7200/7200 [==============================] - 1s 81us/step - loss: 0.4007 - acc: 0.8360
Epoch 90/100
7200/7200 [==============================] - 1s 90us/step - loss: 0.4005 - acc: 0.8347
Epoch 91/100
7200/7200 [==============================] - 1s 86us/step - loss: 0.4007 - acc: 0.8356
Epoch 92/100

7200/7200 [==============================] - 1s 72us/step - loss: 0.3990 - acc: 0.8382
Epoch 64/100
7200/7200 [==============================] - 1s 73us/step - loss: 0.3992 - acc: 0.8369
Epoch 65/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3991 - acc: 0.8361
Epoch 66/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.3990 - acc: 0.8376
Epoch 67/100
7200/7200 [==============================] - 0s 54us/step - loss: 0.3988 - acc: 0.8353
Epoch 68/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3993 - acc: 0.8369
Epoch 69/100
7200/7200 [==============================] - 0s 55us/step - loss: 0.3988 - acc: 0.8360
Epoch 70/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.3991 - acc: 0.8362
Epoch 71/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.3989 - acc: 0.8367
Epoch 72/100
7200/7200 [==============================] - 0s 51us/step - loss: 0.3987 - acc: 0.8358
Epoch 73/100


7200/7200 [==============================] - 0s 54us/step - loss: 0.4032 - acc: 0.8358
Epoch 46/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.4031 - acc: 0.8367
Epoch 47/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.4030 - acc: 0.8336
Epoch 48/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.4029 - acc: 0.8358
Epoch 49/100
7200/7200 [==============================] - 0s 54us/step - loss: 0.4030 - acc: 0.8368: 0s - loss: 0.4142 - acc: 0
Epoch 50/100
7200/7200 [==============================] - 1s 74us/step - loss: 0.4025 - acc: 0.8347
Epoch 51/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4027 - acc: 0.8358
Epoch 52/100
7200/7200 [==============================] - 1s 82us/step - loss: 0.4024 - acc: 0.8351
Epoch 53/100
7200/7200 [==============================] - 0s 56us/step - loss: 0.4021 - acc: 0.8357
Epoch 54/100
7200/7200 [==============================] - 0s 54us/step - loss: 0.4026

7200/7200 [==============================] - 0s 53us/step - loss: 0.3940 - acc: 0.8367
Epoch 27/100
7200/7200 [==============================] - 0s 55us/step - loss: 0.3939 - acc: 0.8357
Epoch 28/100
7200/7200 [==============================] - 0s 54us/step - loss: 0.3934 - acc: 0.8381
Epoch 29/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.3933 - acc: 0.8372
Epoch 30/100
7200/7200 [==============================] - 0s 55us/step - loss: 0.3933 - acc: 0.8361
Epoch 31/100
7200/7200 [==============================] - 0s 52us/step - loss: 0.3928 - acc: 0.8381
Epoch 32/100
7200/7200 [==============================] - 0s 59us/step - loss: 0.3922 - acc: 0.8390
Epoch 33/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3932 - acc: 0.8386
Epoch 34/100
7200/7200 [==============================] - 0s 56us/step - loss: 0.3924 - acc: 0.8369
Epoch 35/100
7200/7200 [==============================] - 0s 53us/step - loss: 0.3920 - acc: 0.8383
Epoch 36/100


...

7200/7200 [==============================] - 1s 77us/step - loss: 0.3368 - acc: 0.8640
Epoch 258/500
7200/7200 [==============================] - 1s 74us/step - loss: 0.3365 - acc: 0.8637
Epoch 259/500
7200/7200 [==============================] - 1s 77us/step - loss: 0.3365 - acc: 0.8646
Epoch 260/500
7200/7200 [==============================] - 1s 75us/step - loss: 0.3370 - acc: 0.8629
Epoch 261/500
7200/7200 [==============================] - 1s 74us/step - loss: 0.3366 - acc: 0.8635
Epoch 262/500
7200/7200 [==============================] - 1s 75us/step - loss: 0.3365 - acc: 0.8632
Epoch 263/500
7200/7200 [==============================] - 1s 76us/step - loss: 0.3363 - acc: 0.8628
Epoch 264/500
7200/7200 [==============================] - 1s 77us/step - loss: 0.3368 - acc: 0.8639
Epoch 265/500
7200/7200 [==============================] - 1s 75us/step - loss: 0.3370 - acc: 0.8628
Epoch 266/500
7200/7200 [==============================] - 1s 78us/step - loss: 0.3366 - acc: 0.8628
Epoc

7200/7200 [==============================] - 1s 150us/step - loss: 0.3358 - acc: 0.8654
Epoch 337/500
7200/7200 [==============================] - 1s 156us/step - loss: 0.3364 - acc: 0.8643
Epoch 338/500
7200/7200 [==============================] - 1s 146us/step - loss: 0.3358 - acc: 0.8637
Epoch 339/500
7200/7200 [==============================] - 1s 152us/step - loss: 0.3362 - acc: 0.8628
Epoch 340/500
7200/7200 [==============================] - 1s 171us/step - loss: 0.3364 - acc: 0.8643
Epoch 341/500
7200/7200 [==============================] - 1s 156us/step - loss: 0.3364 - acc: 0.8632
Epoch 342/500
7200/7200 [==============================] - 1s 161us/step - loss: 0.3362 - acc: 0.8637
Epoch 343/500
7200/7200 [==============================] - 1s 167us/step - loss: 0.3362 - acc: 0.8633
Epoch 344/500
7200/7200 [==============================] - 1s 161us/step - loss: 0.3359 - acc: 0.8637
Epoch 345/500
7200/7200 [==============================] - 1s 152us/step - loss: 0.3360 - acc: 0

7200/7200 [==============================] - 1s 180us/step - loss: 0.3347 - acc: 0.8632
Epoch 417/500
7200/7200 [==============================] - 1s 176us/step - loss: 0.3344 - acc: 0.8654
Epoch 418/500
7200/7200 [==============================] - 1s 163us/step - loss: 0.3347 - acc: 0.8640
Epoch 419/500
7200/7200 [==============================] - 1s 193us/step - loss: 0.3346 - acc: 0.8657
Epoch 420/500
7200/7200 [==============================] - 2s 225us/step - loss: 0.3341 - acc: 0.8656
Epoch 421/500
7200/7200 [==============================] - 2s 219us/step - loss: 0.3345 - acc: 0.8643
Epoch 422/500
7200/7200 [==============================] - 1s 205us/step - loss: 0.3339 - acc: 0.8643
Epoch 423/500
7200/7200 [==============================] - 1s 203us/step - loss: 0.3334 - acc: 0.8653
Epoch 424/500
7200/7200 [==============================] - 1s 200us/step - loss: 0.3345 - acc: 0.8649
Epoch 425/500
7200/7200 [==============================] - 2s 232us/step - loss: 0.3337 - acc: 0

7200/7200 [==============================] - 1s 144us/step - loss: 0.3306 - acc: 0.8647
Epoch 496/500
7200/7200 [==============================] - 1s 161us/step - loss: 0.3308 - acc: 0.8661
Epoch 497/500
7200/7200 [==============================] - 1s 139us/step - loss: 0.3311 - acc: 0.8651
Epoch 498/500
7200/7200 [==============================] - 1s 126us/step - loss: 0.3306 - acc: 0.8640
Epoch 499/500
7200/7200 [==============================] - 1s 122us/step - loss: 0.3309 - acc: 0.8656
Epoch 500/500
7200/7200 [==============================] - 1s 123us/step - loss: 0.3314 - acc: 0.8657
Epoch 1/500
7200/7200 [==============================] - 4s 582us/step - loss: 0.5581 - acc: 0.7960
Epoch 2/500
7200/7200 [==============================] - 1s 143us/step - loss: 0.4341 - acc: 0.7962
Epoch 3/500
7200/7200 [==============================] - 1s 139us/step - loss: 0.4270 - acc: 0.7962
Epoch 4/500
7200/7200 [==============================] - 1s 130us/step - loss: 0.4228 - acc: 0.7962
Ep

7200/7200 [==============================] - 1s 96us/step - loss: 0.3997 - acc: 0.8357
Epoch 77/500
7200/7200 [==============================] - 1s 87us/step - loss: 0.3994 - acc: 0.8353
Epoch 78/500
7200/7200 [==============================] - 1s 85us/step - loss: 0.3994 - acc: 0.8353
Epoch 79/500
7200/7200 [==============================] - 1s 83us/step - loss: 0.3992 - acc: 0.8353
Epoch 80/500
7200/7200 [==============================] - 1s 88us/step - loss: 0.3992 - acc: 0.8339
Epoch 81/500
7200/7200 [==============================] - 1s 89us/step - loss: 0.3996 - acc: 0.8346
Epoch 82/500
7200/7200 [==============================] - 1s 87us/step - loss: 0.3995 - acc: 0.8331
Epoch 83/500
7200/7200 [==============================] - 1s 87us/step - loss: 0.3993 - acc: 0.8344
Epoch 84/500
7200/7200 [==============================] - 1s 85us/step - loss: 0.3992 - acc: 0.8354
Epoch 85/500
7200/7200 [==============================] - 1s 109us/step - loss: 0.3993 - acc: 0.8340
Epoch 86/500

In [ ]:
#get the best parameter and best score

In [ ]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
print('Best parameters are: ', best_parameters)

In [ ]:
print('Best accuracy are: ', best_accuracy)